# Using `jit`

We'll start with a trivial example but get to some more realistic applications shortly.

### Array sum

The function below is a naive `sum` function that sums all the elements of a given array.

In [ ]:
def sum_array(inp):
    J, I = inp.shape
    
    #this is a bad idea
    mysum = 0
    for j in range(J):
        for i in range(I):
            mysum += inp[j, i]
            
    return mysum

In [ ]:
import numpy

In [ ]:
arr = numpy.random.random((300, 300))

In [ ]:
sum_array(arr)

In [ ]:
plain = %timeit -o sum_array(arr)

# Let's get started

In [ ]:
from numba import jit

## As a function call

In [ ]:
sum_array_numba = jit()(sum_array)

What's up with the weird double `()`s?  We'll cover that in a little bit.

In [ ]:
sum_array_numba(arr)

In [ ]:
jitted = %timeit -o sum_array_numba(arr)

In [ ]:
plain.best / jitted.best

## (more commonly) As a decorator

In [ ]:
@jit
def sum_array(inp):
    I, J = inp.shape
    
    mysum = 0
    for i in range(I):
        for j in range(J):
            mysum += inp[i, j]
            
    return mysum

In [ ]:
sum_array(arr)

In [ ]:
%timeit sum_array(arr)

## How does this compare to NumPy?

In [ ]:
%timeit arr.sum()

## When is Numba faster than NumPy?

When doing more complex things, or when using less common integer types, like int16:

In [ ]:
arr_int16 = (arr * 4096).astype(numpy.int16)

In [ ]:
jitted_int16 = %timeit -o sum_array_numba(arr_int16)

In [ ]:
numpy_int16 = %timeit -o arr_int16.sum()

In [ ]:
numpy_int16.best / jitted_int16.best

NumPy doesn't have a specialized version of `sum()` for 16-bit integers, but Numba just generated one that was many times faster!  Numba can take advantage of things like AVX support for packed integers while NumPy has to cast to a larger datatype to use one of the precompiled implementations.

## When does `numba` compile things?

The first time you call the function.  